In [3]:
import os
os.chdir(r"X:\My Projects\SNA\facebook-sna-analysis")

In [4]:
import networkx as nx
file_path = "data/raw/facebook_combined.txt.gz"

G = nx.read_edgelist(file_path, nodetype=int)
print("Number of nodes (actors):", G.number_of_nodes())
print("Number of edges (connections):", G.number_of_edges())
print("Is the graph connected:", nx.is_connected(G))
print("Number of connected components:", nx.number_connected_components(G))


Number of nodes (actors): 4039
Number of edges (connections): 88234
Is the graph connected: True
Number of connected components: 1


In [5]:
density = nx.density(G)
avg_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()

print("Network density:", density)
print("Average degree:", avg_degree)

Network density: 0.010819963503439287
Average degree: 43.69101262688784


In [29]:
degrees = [d for n, d in G.degree()]

In [47]:
import plotly.express as px

fig = px.histogram(
    degrees,
    nbins=50,
    text_auto=True,
    title='Degree Distribution',range_x=(0,1200)
)
fig.update_layout(
    xaxis_title='Degree Range',
    yaxis_title='Frequency'
)
fig.update_traces(
    marker_line_color='black',  # border color
    marker_line_width=1         # border thickness
    
)
fig.show()


In [52]:
fig = px.histogram(
    degrees,
    nbins=100,
    text_auto=True,
    title='Log-scaled Degree Distribution',range_x=(0,400), log_y=True
)
fig.update_layout(
    xaxis_title='Degree Range',
    yaxis_title='Log Frequency'
)
fig.update_traces(
    marker_line_color='black',  # border color
    marker_line_width=1         # border thickness
    
)
fig.show()


In [53]:
import pandas as pd

deg_df = pd.DataFrame(degrees, columns=["degree"])
deg_df.to_csv("outputs/tables/degree_distribution.csv", index=False)

In [61]:
import numpy as np
import pandas as pd

deg = np.array(degrees)
deg_sorted = np.sort(deg)
ccdf = 1.0 - np.arange(1, len(deg_sorted)+1) / len(deg_sorted)

ccdf_df = pd.DataFrame({
    "degree": deg_sorted,
    "ccdf": ccdf
})


fig = px.scatter(
    ccdf_df,
    x="degree",
    y="ccdf",
    log_x=True,
    log_y=True,
    title="Complementary CDF of Degree Distribution"
)
fig.update_layout(
    xaxis_title="Degree (log)",
    yaxis_title="P(K ≥ k) (log)"
)
fig.update_xaxes(range=[0, 3])   # 10^0 to 10^3
fig.update_yaxes(range=[-5, 0])  # 10^-4 to 10^0
fig.show()


In [62]:
import pandas as pd
import networkx as nx

deg_cent = nx.degree_centrality(G)

deg_cent_df = (
    pd.DataFrame(deg_cent.items(), columns=["node", "degree_centrality"])
      .sort_values("degree_centrality", ascending=False)
)

deg_cent_df.head(10)

,node,degree_centrality
107,107,0.258791
351,1684,0.196137
352,1912,0.186974
1821,3437,0.135463
0,0,0.085934
1490,2543,0.072808
2154,2347,0.072065
1373,1888,0.062902
1285,1800,0.060674
1149,1663,0.058197


In [63]:
bet_cent = nx.betweenness_centrality(G, k=500, seed=42)

bet_cent_df = (
    pd.DataFrame(bet_cent.items(), columns=["node", "betweenness"])
      .sort_values("betweenness", ascending=False)
)

bet_cent_df.head(10)

,node,betweenness
107,107,0.489707
351,1684,0.344536
1821,3437,0.222233
352,1912,0.216324
0,0,0.161446
571,1085,0.150297
1843,698,0.113607
1710,567,0.088151
58,58,0.082741
350,428,0.060826


In [64]:
close_cent = nx.closeness_centrality(G)

close_cent_df = (
    pd.DataFrame(close_cent.items(), columns=["node", "closeness"])
      .sort_values("closeness", ascending=False)
)

close_cent_df.head(10)

,node,closeness
107,107,0.459699
58,58,0.397402
350,428,0.394837
371,563,0.393913
351,1684,0.393606
171,171,0.370493
348,348,0.369916
366,483,0.369848
349,414,0.369543
362,376,0.366558


In [71]:
centrality_df = pd.DataFrame({
    "node": list(G.nodes()),
    "degree_centrality": [deg_cent[n] for n in G.nodes()],
    "betweenness": [bet_cent[n] for n in G.nodes()],
    "closeness": [close_cent[n] for n in G.nodes()]
})
centrality_df.sort_values("degree_centrality", ascending=False).head(10)


,node,degree_centrality,betweenness,closeness
107,107,0.258791,0.489707,0.459699
351,1684,0.196137,0.344536,0.393606
352,1912,0.186974,0.216324,0.350947
1821,3437,0.135463,0.222233,0.314413
0,0,0.085934,0.161446,0.353343
1490,2543,0.072808,0.006708,0.291300
2154,2347,0.072065,0.002758,0.283408
1373,1888,0.062902,0.000139,0.321292
1285,1800,0.060674,0.000216,0.321599
1149,1663,0.058197,0.001180,0.339185


In [73]:
centrality_df.to_csv("outputs/tables/facebook_centrality_metrics.csv", index=False)

In [75]:
import plotly.express as px

fig = px.scatter(
    centrality_df,
    x="degree_centrality",
    y="betweenness",
    log_x=True,
    log_y=True,
    title="Degree vs Betweenness Centrality (Log–Log)",
    labels={
        "degree_centrality": "Degree Centrality (log)",
        "betweenness": "Betweenness Centrality (log)"
    }
)

fig.update_traces(marker=dict(size=6, opacity=0.6))
fig.show()

In [78]:
fig = px.histogram(
    centrality_df,
    x="degree_centrality",
    nbins=100,
    title="Distribution of Degree Centrality"
)

fig.update_layout(
    xaxis_title="Degree Centrality",
    yaxis_title="Frequency"
)
fig.update_traces(
    marker_line_color='black',  # border color
    marker_line_width=1         # border thickness
    
)
fig.show()

In [79]:
fig = px.histogram(
    centrality_df,
    x="betweenness",
    nbins=50,
    title="Distribution of Betweenness Centrality"
)

fig.update_yaxes(type="log")
fig.update_layout(
    xaxis_title="Betweenness Centrality",
    yaxis_title="Log Frequency"
)
fig.update_traces(
    marker_line_color='black',  # border color
    marker_line_width=1         # border thickness
    
)
fig.show()


In [80]:
top_nodes = (
    centrality_df
    .sort_values("betweenness", ascending=False)
    .head(100)["node"]
    .tolist()
)

subG = G.subgraph(top_nodes)

In [83]:
import plotly.graph_objects as go

pos = nx.spring_layout(subG, seed=42)

edge_x = []
edge_y = []

for edge in subG.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

node_x = []
node_y = []
node_size = []

for node in subG.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_size.append(
        centrality_df.loc[
            centrality_df["node"] == node,
            "betweenness"
        ].values[0] * 5000
    )

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=node_size,
        color=node_size,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Betweenness")
    ),
    text=[f"Node {n}" for n in subG.nodes()]
)

fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title="Top 100 Facebook Users by Betweenness Centrality",
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig.show()


In [88]:
import community as community_louvain
largest_cc = max(nx.connected_components(G), key=len)
G_cc = G.subgraph(largest_cc)
partition = community_louvain.best_partition(G_cc)
num_communities = len(set(partition.values()))
num_communities

15

In [89]:
modularity = community_louvain.modularity(partition, G_cc)
modularity

0.8348446762780308

In [90]:
community_df = pd.DataFrame.from_dict(partition, orient="index", columns=["community"])
community_sizes = community_df["community"].value_counts()

community_sizes.head(10)


community
4     554
14    548
2     447
1     429
3     423
0     350
5     326
10    237
7     226
13    206
Name: count, dtype: int64

In [ ]:
top_comms = community_sizes.head(10).index.tolist()

nodes_top = community_df[
    community_df["community"].isin(top_comms)
].index.tolist()

subG = G_cc.subgraph(nodes_top)

In [91]:
import plotly.graph_objects as go

pos = nx.spring_layout(subG, seed=42)

edge_x, edge_y = [], []
for u, v in subG.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    mode="lines",
    line=dict(width=0.4, color="#aaa"),
    hoverinfo="none"
)

node_x, node_y, colors = [], [], []
for n in subG.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    colors.append(partition[n])

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode="markers",
    marker=dict(
        size=6,
        color=colors,
        colorscale="Turbo",
        showscale=True,
        colorbar=dict(title="Community ID")
    ),
    hoverinfo="text",
    text=[f"Node {n}<br>Community {partition[n]}" for n in subG.nodes()]
)

fig = go.Figure([edge_trace, node_trace])
fig.update_layout(
    title="Facebook Network Communities (Top 10 Communities)",
    showlegend=False,
    hovermode="closest"
)

fig.show()


In [93]:
centrality_df["community"] = centrality_df["node"].map(partition)

fig = px.box(
    centrality_df.dropna(),
    x="community",
    y="betweenness",
    title="Betweenness Centrality Across Communities"
)

fig.update_yaxes(type="log")
fig.show()